# Multiple Lenses Sequence

A more complex example of a system with multiple lenses in sequence. The number of lenses of each type is a easy to change parameter.

In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from itertools import islice, repeat


from torchlensmaker.optics import FocalPointLoss, ParallelBeamUniform, ParallelBeamRandom, FixedGap, RefractiveSurface, OpticalStack, Lens, Anchor
from torchlensmaker.shapes import Parabola, PiecewiseLine, Line
from torchlensmaker.training import render, optimize


class PulaskiStack(nn.Module):
    def __init__(self, lens_radius, focal_length, lens_min_thickness, nplano, nbiconvex, nrplano):
        super().__init__()
        
        plane = Line(lens_radius, [0.0, 1.0, 0.0], optimize=False)
        convex = Parabola(lens_radius, [-0.002])
        rconvex = convex.share(scale=-1)
    
        biconvex1 = Parabola(lens_radius, [0.002])
        biconvex2 = biconvex1.share(scale=-1)
    
        self.plano = Lens(
            RefractiveSurface(plane, (1.0, 1.49), anchors=(Anchor.Center, Anchor.Edge)),
            FixedGap(lens_min_thickness),
            RefractiveSurface(convex, (1.49, 1.0), anchors=(Anchor.Edge, Anchor.Center)),
        )
        
        self.biconvex = Lens(
            RefractiveSurface(biconvex1, (1.0, 1.49), anchors=(Anchor.Center, Anchor.Edge)),
            FixedGap(lens_min_thickness),
            RefractiveSurface(biconvex2, (1.49, 1.0), anchors=(Anchor.Edge, Anchor.Center)),
        )
    
        self.rplano = Lens(
            RefractiveSurface(rconvex, (1.0, 1.49), anchors=(Anchor.Center, Anchor.Edge)),
            FixedGap(lens_min_thickness),
            RefractiveSurface(plane, (1.49, 1.0), anchors=(Anchor.Edge, Anchor.Center)),
        )

        self.stack = OpticalStack([
            ParallelBeamUniform(radius=1.0*lens_radius),
            FixedGap(10.),
            *[
                FixedGap(lens_spacing),
                self.plano,
            ]*nplano,
            *[
                FixedGap(lens_spacing),
                self.biconvex,
            ] *nbiconvex,
            *[
                FixedGap(lens_spacing),
                self.rplano,
            ]*nrplano,
            FixedGap(focal_length),
            FocalPointLoss(),
        ])

    def forward(self, inputs, hook=None):
        return self.stack.forward(inputs, hook=hook)
        

torch.set_printoptions(precision=10)

def parabolic_lens_thickness(lens, r):
    "Total thickness of parabolic lens at distance r from the center"

    y1 = lens.surface1.surface.evaluate(r)[0, 1].item()
    y2 = lens.surface2.surface.evaluate(r)[0, 1].item()
    return lens.thickness()[0] - y1 + y2


square_size = 30
lens_radius = math.sqrt(2)*square_size/2
focal_length = 55.
nplano = 1
nbiconvex = 3
nrplano = 2
lens_min_thickness = 1.2
lens_spacing = 3.


def regu_equalparam(optics):
    params = torch.cat([param.view(-1) for param in optics.parameters()])
    return torch.pow(torch.diff(1000*torch.abs(params)).sum(), 2)

def regu_equalthickness(optics):
    t0 = optics.plano.thickness()[0]
    t1 = optics.biconvex.thickness()[0]
    return 100*torch.pow(t0 - t1, 2)

def demo_pulaski():
    print("Pulaski design")
    print("Square size", square_size)
    print("Lens radius", lens_radius)
    print("Configuration", nplano, nbiconvex, nrplano)
    print("lens_min_thickness", lens_min_thickness)
    print("lens_spacing", lens_spacing)

    optics = PulaskiStack(lens_radius, focal_length, lens_min_thickness, nplano, nbiconvex, nrplano)
    lens_plano, lens_biconvex, lens_rplano = optics.plano, optics.biconvex, optics.rplano
    
    render(optics, num_rays=10, force_uniform_source=False)
        

    optimize(
        optics,
        optimizer = optim.Adam(optics.parameters(), lr=1e-4),
        num_rays = 35,
        num_iter = 250,
        regularization = regu_equalthickness,
    )

    render(optics, num_rays=10)

    # Thickness profile
    half_square_size = square_size/2
    
    def thickness_profile(lens):
        a, c = lens.thickness()
        b = parabolic_lens_thickness(lens, square_size/2)
        return a, b, c

    print("Plano-convex thickness {:.4f} {:.4f} {:.4f}".format(*thickness_profile(lens_plano)))
    print("Bi-convex thickness {:.4f} {:.4f} {:.4f}".format(*thickness_profile(lens_biconvex)))
    print("Reverse plano-convex thickness {:.4f} {:.4f} {:.4f}".format(*thickness_profile(lens_rplano)))
    print()
    print(list(optics.parameters()))

    return optics, lens_plano, lens_biconvex, lens_rplano

optics, lens_plano, lens_biconvex, lens_rplano = demo_pulaski()

In [ ]:
from torchlensmaker.optics import Lens
from torchlensmaker.export3d import lens_to_part

from IPython.display import display

display(lens_to_part(optics.plano))
display(lens_to_part(optics.biconvex))